In [199]:
from EvGym.charge_world import ChargeWorldEnv, Session
from EvGym.charge_agent import agentRB
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
from IPython.display import clear_output
from EvGym import config
import datetime
from math import isclose

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [3]:
df_data = pd.read_csv("data/prepared_elaad_transactions.csv", parse_dates=["TransactionStartDT", "TransactionStopDT"])
df_data["TransactionStartDate"] = pd.to_datetime(df_data["TransactionStartDate"]).dt.date
df_data["TransactionStopDate"] = pd.to_datetime(df_data["TransactionStopDate"]).dt.date
print(f"TransactionStartTS:\t min {df_data['TransactionStartTS'].min()},\tmax {df_data['TransactionStartTS'].max()}")
print(f"TransactionStopTS:\t min {df_data['TransactionStopTS'].min()},\tmax {df_data['TransactionStopTS'].max()}")
print("--------")
df_data.info()

TransactionStartTS:	 min 1.0,	max 35028.0
TransactionStopTS:	 min 33.0,	max 35104.0
--------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   TransactionId            10000 non-null  int64         
 1   ChargePoint              10000 non-null  object        
 2   Connector                10000 non-null  int64         
 3   TransactionStartDT       10000 non-null  datetime64[ns]
 4   TransactionStopDT        10000 non-null  datetime64[ns]
 5   StartCard                10000 non-null  object        
 6   ConnectedTime            10000 non-null  float64       
 7   ChargeTime               10000 non-null  float64       
 8   TotalEnergy              10000 non-null  float64       
 9   MaxPower                 10000 non-null  float64       
 10  BatteryCapacity          10000 non-null  int64         
 11  S

In [4]:
df_data[df_data["TransactionStartTS"] == 1].sort_values(["TransactionStartTS"]).head()

,TransactionId,ChargePoint,Connector,TransactionStartDT,TransactionStopDT,StartCard,ConnectedTime,ChargeTime,TotalEnergy,MaxPower,...,TransactionStopSecDay,TransactionStartHourDay,TransactionStopHourDay,TransactionStartAbsSec,TransactionStopAbsSec,TransactionStartTS,TransactionStopTS,ConnectedTimeMin,measConnectedTimeMin,errConnectedTime
0,3261657,e62c50d1be0a2f80ec51d471f9630a4e,2,2019-01-01 00:30:00,2019-01-01 08:24:00,826d337c1d84900452edfcb27a3fb11bcbd06f35da8406abd9ca28b1291d219c,7.91,1.0,6.53,9.818,...,30240.0,0.0,8.0,0.0,28440.0,1.0,33.0,474.6,474.0,0.6


In [4]:
world = ChargeWorldEnv(df_data)
agent = agentRB()


In [5]:

df_state = world.reset()
df_state_minus1 = df_state.copy()
for i in range(df_data.shape[0]):
    clear_output(wait=True)  # Clear the output from the previous iteration
    action = agent.get_action(df_state)
    df_state, reward, done, info = world.step(action)
    
    display(df_state)  # Display the data frame
    
    #display(f"t= {world.t}, action {action}, power {world.power_t.sum()}")  # Display the iteration number
    
    if world.df_depart.shape[0] > 0:
        for _, dep in world.df_depart.iterrows():
            print(f"{dep.idSess} departed with soc of {dep.soc_t}")
    print(f"t= {world.t}, reward {reward},\npower {world.power_t.sum():.02f}, \naction {action[:10]}\n{action[10:]}")
    
    if (df_state_minus1.values == df_state.values).all():
        time.sleep(0.2)
    else:
        input()
    df_state_minus1 = df_state.copy()


    

,idSess,t_arr,soc_arr,t_dep,soc_t
0,-1,0,0,0,0
1,-1,0,0,0,0
2,-1,0,0,0,0
3,-1,0,0,0,0
4,-1,0,0,0,0
5,-1,0,0,0,0
6,-1,0,0,0,0
7,-1,0,0,0,0
8,-1,0,0,0,0
9,-1,0,0,0,0


3261657 departed with soc of 1
t= 33, reward 0,
power 0.00, 
action [0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000]


KeyboardInterrupt: Interrupted by user

In [ ]:
world.reset()
# Before 50-60 sec
for i in range(10): #tqdm(range(df_data["TransactionStopTS"].max().astype(int))):
    world.step(np.zeros(world.max_cars))

#world.step()
#print(world.t)
#world.df_park.head(200)


In [12]:
type(df_data.values)

numpy.ndarray

In [119]:
df_c = pd.DataFrame([[0,1,'a'], [1, 2, 'b'], [1, 3, 'c'], [0,4, 'd']], columns = ["id", "age", "letter"])
big_id = df_c["id"] > 0
df_c.loc[big_id, "age"] = 2*df_c.loc[big_id]["age"] + df_c.loc[big_id]["id"]
df_c

,id,age,letter
0,0,1,a
1,1,5,b
2,1,7,c
3,0,4,d


In [125]:
~big_id

0     True
1    False
2    False
3     True
Name: id, dtype: bool

In [61]:
df_a_lag = pd.DataFrame([[0,'a'], [0, 'b'], [0, 'c'], [0, "d"]], columns = ["id", "letter"])
df_a = pd.DataFrame([[0,'a'], [1, 'b'], [0, 'c'], [1, "d"]], columns = ["id", "letter"])

In [66]:
new_arrival_id = df_a['id'] != df_a_lag['id']
arr_a = df_a.values
for i, row in df_a.iterrows():
    string = str(i) + " " + row["letter"]
    if new_arrival_id[i]:
        string = config.bcolors.FAIL + string + config.bcolors.ENDC
    print(string)

0 a
1 m
2 c
3 m


In [67]:
df_a.iloc[0].letter

'a'

In [68]:
blank_row = [0, "m"]

In [69]:
display(df_a)
index_replace = (df_a["id"] == 1)
print(index_replace)
df_a.loc[index_replace] = blank_row
df_a

,id,letter
0,0,a
1,0,m
2,0,c
3,0,m


0    False
1    False
2    False
3    False
Name: id, dtype: bool


,id,letter
0,0,a
1,0,m
2,0,c
3,0,m


In [100]:
arr_num = np.array([1,2,3,4])
(arr_num[barr_big] > 2).all()

False

In [102]:
arr_clip = np.clip(arr_num, 2, 3)
arr_clip

array([2, 2, 3, 3])

In [94]:
barr_big.any()

True

In [86]:
barr_big = df_a["id"] > 1
df_a.loc[index_big, "id"] += 1

df_a

,id,letter
0,1,a
1,32,m
2,32,c
3,1,m


In [79]:
for i, row in df_a.iterrows():
    df_a.loc[[1,2], "id"] += 1
df_a

,id,letter
0,1,a
1,25,m
2,25,c
3,1,m


In [73]:
pd._typing.F

~F

In [135]:
datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S")

'2023-03-29-12:46:19'

In [186]:
dict_a = {0: 0, 1: 11, 2: 22, 3: 33}
dict_b = {1: 101, 2: 202, 3: 303, 4: 404}

df_a = pd.DataFrame(np.array([list(dict_a.keys()), (list(dict_a.values()))]).T, columns = ["time", "val_a"])
df_b = pd.DataFrame(np.array([list(dict_b.keys()), (list(dict_b.values()))]).T, columns = ["time", "val_b"])

In [187]:
df_a

,time,val_a
0,0,0
1,1,11
2,2,22
3,3,33


In [195]:
df_res = pd.merge(df_a, df_b, on = ["time"], how= "outer")
df_res = df_res.fillna(0)
df_res

,time,val_a,val_b
0,0,0.0,0.0
1,1,11.0,101.0
2,2,22.0,202.0
3,3,33.0,303.0
4,4,0.0,404.0


In [197]:
df_res = pd.DataFrame([[1, 11, 21], [2, 12, 22], [3, 13, 23]], columns = ["A", "B", "C"] )
df_res

,A,B,C
0,1,11,21
1,2,12,22
2,3,13,23


In [201]:
B_list = [11, 12, 13]
np.isclose(df_res["B"], B_list)

array([ True,  True,  True])

In [203]:
np.arange(0, 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])